# Solving Stokes in parallel

In [ ]:
from ipyparallel import Cluster
c = await Cluster(engines="mpi").start_and_connect(n=4, activate=True)

In [ ]:
%%px
from mpi4py import MPI
from ngsolve import *
from netgen.occ import *
from ngsolve.krylovspace import CGSolver
from ngsolve.krylovspace import BramblePasciakCG

box = Box((0,0,0), (2.5,0.41,0.41))
box.faces.name="wall"
box.faces.Min(X).name="inlet"
box.faces.Max(X).name="outlet"
cyl = Cylinder((0.5,0.2,0), Z, h=0.41,r=0.05)
cyl.faces.name="cyl"
shape = box-cyl

ngmesh = OCCGeometry(shape).GenerateMesh(maxh=0.05, comm=MPI.COMM_WORLD)

for r in range(1): ngmesh.Refine()
mesh = Mesh(ngmesh)
print (mesh.GetNE(VOL))

In [ ]:
%%px
# import ngsolve.ngs2petsc as n2p
import ngsPETSc as n2p
import petsc4py.PETSc as psc

In [ ]:
%%px
V = VectorH1(mesh, order=1, dirichlet="wall|inlet|cyl")
V1 = H1(mesh, order=1, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order=1)
if mesh.comm.rank==0:
    print ("ndof = ", V.ndofglobal,'+',Q.ndofglobal,'=',
           V.ndofglobal+Q.ndofglobal)

u,v = V.TnT()
u1,v1 = V1.TnT()
p,q = Q.TnT()

h = specialcf.mesh_size

bfa1 = BilinearForm(InnerProduct(grad(u1),grad(v1))*dx)
bfb = BilinearForm(div(u)*q*dx).Assemble()
bfc = BilinearForm(h*h*grad(p)*grad(q)*dx).Assemble()

prea1 = Preconditioner(bfa1, "PETScPC")
bfa1.Assemble()

mata = sum( [Ri.T@bfa1.mat@Ri for Ri in V.restrictions] )
prea = sum( [Ei@prea1@Ei.T for Ei in V.embeddings])    
    
bfschur = BilinearForm(p*q*dx, diagonal=True).Assemble()
preschur = bfschur.mat.Inverse()

In [ ]:
%%px

gfu = GridFunction(V)
gfp = GridFunction(Q)

uin = (1.5*4*y*(0.41-y)/(0.41*0.41)*z*(0.41-z)/0.41**2,0, 0)

gfu.Set(uin, definedon=mesh.Boundaries("inlet"))

resf = (-mata * gfu.vec).Evaluate()
resg = (-bfb.mat * gfu.vec).Evaluate()

sol = BramblePasciakCG (A=mata, B=bfb.mat, C=bfc.mat, f=resf, g=resg, \
                preA=prea, preS=preschur, maxit=500, 
                printrates='\r' if mesh.comm.rank==0 else False)

gfu.vec.data += sol[0]
gfp.vec.data += sol[1]

In [ ]:
gfu = c[:]["gfu"]

In [ ]:
from ngsolve.webgui import Draw
Draw (gfu[0], clipping={"y":0, "z":-1});

In [ ]:
gfp = c[:]["gfp"]
Draw (gfp[0]);